In [2]:
from qiskit.quantum_info import Pauli
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
import numpy as np
from itertools import product
import plotly.express as px

%load_ext autoreload
%autoreload 2

In [3]:
num_qubit = 4
k_loc =2

basisL = KLocalPauliBasis(k_loc,num_qubit)
basisC = KLocalPauliBasis(k_loc+1,num_qubit)
basisS = KLocalPauliBasis(2*k_loc,num_qubit)

def commutator(pauli1_str, pauli2_str):
    pauli1 = Pauli(pauli1_str)
    pauli2 = Pauli(pauli2_str)
    if pauli1.commutes(pauli2):
        return None
    else:
        operator = 1j * (pauli1 @ pauli2)
        phase = 1j**operator.phase
        pauli_label = (operator * phase).to_label()
        # sign = '+' if phase==1 else '-'
        return pauli_label
    
def fun(i,j):
    return commutator(basisC.paulis_list[int(i)],basisL.paulis_list[int(j)])

K = np.zeros((basisC.size,basisL.size),dtype=object)
for i,j in product(range(basisC.size),range(basisL.size)):
    K[i,j] = fun(i,j)
print(K[:10,:10])
print(K.shape)

[[None None None None 'ZIII' None None None 'YIII' None]
 [None None None None None 'IZII' None None None 'IYII']
 [None None None None None None 'IIZI' None None None]
 [None None None None None None None 'IIIZ' None None]
 ['ZIII' None None None None None None None 'XIII' None]
 [None 'IZII' None None None None None None None 'IXII']
 [None None 'IIZI' None None None None None None None]
 [None None None 'IIIZ' None None None None None None]
 ['YIII' None None None 'XIII' None None None None None]
 [None 'IYII' None None None 'IXII' None None None None]]
(111, 39)


In [31]:
from tqdm import tqdm
counter = {}
num_qubit = 8
k_loc =4

basisL = KLocalPauliBasis(k_loc,num_qubit)
basisC = KLocalPauliBasis(k_loc+1,num_qubit)

for c,l in tqdm(list(product(basisC.paulis_list,basisL.paulis_list))):
    comm = commutator(c,l)
    if comm in counter:
        counter[comm] +=1
    elif comm is not None:
        counter[comm] = 1
        


 45%|████▍     | 1519644/3403521 [04:36<05:42, 5501.51it/s]


KeyboardInterrupt: 

In [5]:
counter = {}
num_qubit = 8
k_loc = 4

basisL = KLocalPauliBasis(k_loc,num_qubit)
basisC = KLocalPauliBasis(k_loc+1,num_qubit)



import concurrent.futures

# Define your function to count the frequency of answers
def count_answers(answer_list):
    answer_count = {}
    for a in answer_list:
        answer = commutator(*a)
        if answer in answer_count:
            answer_count[answer] += 1
        elif answer is not None:
            answer_count[answer] = 1
    return answer_count

# Define your list of answers
answers = list(product(basisC.paulis_list,basisL.paulis_list))

# Create a ThreadPoolExecutor to execute the function in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Divide the list of answers into smaller chunks
    chunk_size = len(answers) // 64
    answer_chunks = [answers[i:i+chunk_size] for i in range(0, len(answers), chunk_size)]
    
    # Submit the function to the executor for each chunk of answers
    futures = [executor.submit(count_answers, answer_chunk) for answer_chunk in answer_chunks]
    
    # Combine the results from each chunk into a final answer count
    answer_count = {}
    for future in concurrent.futures.as_completed(futures):
        chunk_answer_count = future.result()
        for answer, count in chunk_answer_count.items():
            if answer in answer_count:
                answer_count[answer] += count
            else:
                answer_count[answer] = count
                
# Print the final answer count
frequencies = list(answer_count.values())
print(len(frequencies),sum(frequencies),sum(frequencies)/len(frequencies))

In [27]:
frequencies = list(counter.values())
print(len(frequencies),sum(frequencies),sum(frequencies)/len(frequencies))

49501 843049 17.030948869719804


In [4]:
px.imshow((K!=None).T).show()
print(np.sum(K!=None)/K.size)

0.47124047124047125


In [3]:
E = np.empty((basisC.size,basisL.size,basisS.size),dtype=float)

def commutator_onehot(pauli1_str, pauli2_str):
    pauli1 = Pauli(pauli1_str)
    pauli2 = Pauli(pauli2_str)
    pauli_index = np.zeros(basisS.size,dtype=float)
    if pauli1.anticommutes(pauli2):
        operator = 1j * (pauli1 @ pauli2)
        phase = 1j**operator.phase
        pauli_label = (operator * phase).to_label()
        # sign = '+' if phase==1 else '-'
        pauli_index[basisS.pauli_to_num(pauli_label)] = np.random.uniform(-1,1)
    return pauli_index

for i,j in product(range(basisC.size),range(basisL.size)):
    E[i,j] = commutator_onehot(basisC.paulis_list[int(i)],basisL.paulis_list[int(j)])
print(E[:10,:10])
print(E.shape)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
(207, 63, 

In [4]:
from scipy.linalg import cholesky,solve_triangular
print(E.shape,basisL.size)
A = np.random.uniform(-1,1,size=(207,63))
def timefun():
    x = np.random.uniform(size=basisL.size)
    covmat = np.einsum("kio,i,j,ljo -> kl",E,x,x,E,optimize=True)
    chol = cholesky(covmat,lower=True)
    c = solve_triangular(chol, A@x)
    return c
%timeit timefun()

(207, 63, 639) 63
26.4 ms ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from gibbs.dataclass import get_results
res = get_results("../saved_simulations/turbo/random1localcfields")[0]
cmatfac = ConstraintMatrixFactory(res.num_qubits,res.klocality,res.klocality+1)

In [6]:
K,E = cmatfac.create_cmat(res.state_vector(-1),100)